In [1]:
# Make sure the following packages are installed as we rely on limmbo for covariance estimation!
#pip3 install "limix<2"


import pandas as pd
from scipy import linalg
import torch
from sklearn import linear_model

In [2]:
import numpy as np
from sklearn.metrics import mean_squared_error

#import model
from imp import reload
#reload model

In [3]:
from pkg_resources import resource_filename
from limmbo.io.reader import ReadData
from limmbo.io.input import InputData

In [4]:
from limmbo.core.vdsimple import vd_reml   

In [6]:
from regmtlmm.twclust import twclust

In [10]:
#Reading input data: phenotype matrix, genotype matrix, relatedness matrix 

data = ReadData(verbose=False)

file_pheno = resource_filename('limmbo','io/test/data/pheno.csv')
file_geno = resource_filename('limmbo', 'io/test/data/genotypes.csv')
file_relatedness = resource_filename('limmbo','io/test/data/relatedness.csv')

In [11]:
data.getPhenotypes(file_pheno=file_pheno)
data.getRelatedness(file_relatedness=file_relatedness,delim=",")
data.getGenotypes(file_genotypes=file_geno)

In [12]:
indata = InputData(verbose=False)
indata.addPhenotypes(phenotypes = data.phenotypes)

In [13]:
relatedness=pd.io.parsers.read_csv(file_relatedness,sep=',')

In [14]:
relatedness.index = relatedness.columns

In [15]:
indata.addRelatedness(relatedness = relatedness)

In [16]:
indata.addGenotypes(genotypes=data.genotypes,genotypes_info=data.genotypes_info)
indata.commonSamples()
#indata.transform(transform="scale")

In [17]:
#Getting eigenvectors and eigenvalues of relatedness matrix
from numpy import linalg as LA
indata.S_R, indata.U_R = LA.eig(relatedness)



In [18]:
n=indata.phenotypes.shape[0]  #sample size
q=indata.phenotypes.shape[1]  #num phenotypes
p=indata.genotypes.shape[1]   #num genotypes

In [19]:
q

10

In [20]:
#computing kronecker product of identity with genotype matrix
x= indata.genotypes.to_numpy()
X= np.kron(np.identity(q),x)

#vectorizing the phenotype matrix
Y=indata.phenotypes.to_numpy()
y=np.matrix.flatten(Y,'F')

# mtLMM-L1

In [21]:
maxiter=1


for i in range(maxiter):
    print('Iteration {}'.format(i))
    # 1) Given fixed effects, estimate covariance matrices based on standard REML
    #for larger number of phenotypes, use limmbo bootstrapped version instead
    Cg, Cn, ptime = vd_reml(indata, verbose=False)
    
    # 2) Given covariance matrices, fit fixed effects using L1 regularization
    H=np.kron(indata.relatedness,Cg)+np.kron(np.ones((n,n)),Cn)
    H_cpu = torch.tensor(H, device='cpu')
    Hinv_cpu = torch.inverse(H_cpu)
    Hinv_cpu_s = torch.cholesky(Hinv_cpu)
    Hinv_s = Hinv_cpu_s.numpy()
    
    ytilde=Hinv_s @ y
    Xtilde=Hinv_s @ X
    
    # running lasso regression with desired sparsity level (here alpha=0.2)
    clf=linear_model.Lasso(alpha=0.2,fit_intercept=False)
    clf.fit(X=Xtilde,y=ytilde)
    
    beta=np.reshape(clf.coef_,(p,q),'F')
    
    #update residuals for new cova estimation step
    newY=Y-x@beta
    indata.phenotypes= pd.DataFrame(newY, index=indata.pheno_samples,
                                       columns=indata.phenotype_ID)



Iteration 0


In [49]:
myprocn

248.879183

# mtLMM-clust

In [16]:
maxiter=10
for i in range(maxiter):
    print('Iteration {}'.format(i))
    #Given fixed effects, estimate covariance matrices based on standard REML
    #for larger number of phenotypes, use limmbo bootstrapped version instead
    Cg, Cn, ptime = vd_reml(indata, verbose=False)
    
    #Given covariance matrices, fit fixed effects using L1 regularization
    H=np.kron(indata.relatedness,Cg)+np.kron(np.ones((n,n)),Cn)
    H_cpu = torch.tensor(H, device='cpu')
    Hinv_cpu = torch.inverse(H_cpu)
    Hinv_cpu_s = torch.cholesky(Hinv_cpu)
    Hinv_s = Hinv_cpu_s.numpy()
    
    ytilde=Hinv_s @ y
    Xtilde=Hinv_s @ X
    
    #running regularized estination with trait-wise clustering 
    estimator = twclust(p=p, n=n, K=q)
    theta_hat = estimator.fit(Xtilde, ytilde, C1=1000, C2=2000, gamma=0.003, maxiter=1000)
    beta = estimator.Theta
    
    #update residuals for new cova estimation step
    newY=Y-x@beta
    indata.phenotypes= pd.DataFrame(newY, index=indata.pheno_samples,
                                       columns=indata.phenotype_ID)



Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9


In [17]:
#displaying pxq fixed effect matrix
beta

array([[ 1.04123795e-02, -1.04082882e-03, -8.06553435e-02,
        -1.30223443e-01,  1.27443810e-01, -2.23887714e-02,
         9.69707630e-02, -4.32734895e-02,  1.96394280e-03,
         4.34679012e-02],
       [ 1.47395009e-03, -2.79325743e-01,  5.26374363e-02,
         1.24200115e-01, -1.53963775e-01, -2.60243156e-02,
         1.06121736e-02,  2.08744732e-01,  1.38817209e-01,
        -1.92121278e-01],
       [ 3.19078419e-02, -6.40377066e-04, -4.26615789e-02,
        -1.91488667e-01,  2.29847538e-01, -6.07065946e-02,
         1.61031452e-01, -4.31601636e-02,  6.69850911e-02,
         5.56273170e-03],
       [-9.93713088e-02,  3.40074922e-01, -1.50365492e-01,
         9.90726279e-02, -3.96781341e-01,  3.37294665e-02,
         1.77614669e-01, -1.61948888e-01,  2.55481949e-01,
        -2.91253189e-01],
       [-6.01146353e-02,  1.25278072e-01, -1.55196474e-01,
         8.95903716e-02, -2.37234581e-01,  1.44358366e-01,
         8.58287845e-02, -3.37939094e-02, -3.06732944e-02,
        -1.

# testing stuff

In [16]:
Cg, Cn, ptime = vd_reml(indata, verbose=False)
    
#Given covariance matrices, fit fixed effects using L1 regularization
H=np.kron(indata.relatedness,Cg)+np.kron(np.ones((n,n)),Cn)
H_cpu = torch.tensor(H, device='cpu')
Hinv_cpu = torch.inverse(H_cpu)
Hinv_cpu_s = torch.cholesky(Hinv_cpu)
Hinv_s = Hinv_cpu_s.numpy()
    
ytilde=Hinv_s @ y
Xtilde=Hinv_s @ X

In [17]:
estimator = twclust(p=p, n=n, K=q)

In [18]:
theta_hat = estimator.fit(Xtilde, ytilde, C1=1000, C2=2000, gamma=0.003)
    

In [19]:
estimator.Theta

array([[ 0.0554274 , -0.07064992, -0.00136169, -0.17422341,  0.16169661,
        -0.00417551, -0.00143251, -0.05476136, -0.0025866 ,  0.07545721],
       [-0.02877624, -0.24460239,  0.18237792,  0.12683426, -0.1461757 ,
        -0.03831001,  0.0352806 ,  0.21527217,  0.11132022, -0.25816572],
       [ 0.03576174, -0.03121809, -0.13155619, -0.11403586,  0.24266137,
        -0.06315243,  0.19213139,  0.0047201 , -0.0021846 , -0.04938717],
       [-0.0602393 ,  0.48084305, -0.24229579,  0.08899356, -0.18095254,
         0.03391957,  0.19433634, -0.18623733,  0.2108622 , -0.3142058 ],
       [-0.08270499,  0.08883558, -0.03401128,  0.05847103, -0.21126979,
         0.09929158,  0.03189868, -0.03564277,  0.00235162, -0.15936019],
       [ 0.24595333,  0.10007849,  0.34085257, -0.1082861 ,  0.07109789,
        -0.06586149,  0.10384288,  0.02085662, -0.0647848 ,  0.16676563],
       [-0.049959  , -0.09912122,  0.01601177,  0.06072197, -0.08166642,
         0.03114192, -0.26486665,  0.11652537

## Testing stuff

In [13]:

##########################

Cg, Cn, ptime = vd_reml(indata, verbose=False)
#gwas = GWAS(datainput=indata, verbose=False)

In [14]:
Cg

array([[ 0.03847333,  0.00289154, -0.03833569,  0.0502919 , -0.00140568,
        -0.04882837,  0.044608  ,  0.02917599, -0.01811099, -0.0144089 ],
       [ 0.00289154,  0.06364759, -0.02286691,  0.01153651,  0.02850815,
         0.00206946, -0.02808302, -0.02756487,  0.01988609, -0.02490405],
       [-0.03833569, -0.02286691,  0.0802884 , -0.09324748, -0.04987963,
         0.05074036, -0.05887076, -0.05307189,  0.02378802,  0.05403899],
       [ 0.0502919 ,  0.01153651, -0.09324748,  0.12159661,  0.05670872,
        -0.07852476,  0.08452257,  0.08090183, -0.0353503 , -0.05516368],
       [-0.00140568,  0.02850815, -0.04987963,  0.05670872,  0.07943548,
        -0.0050005 ,  0.01398618,  0.03497332, -0.00546859, -0.05513013],
       [-0.04882837,  0.00206946,  0.05074036, -0.07852476, -0.0050005 ,
         0.0803708 , -0.0686989 , -0.06395477,  0.02976621,  0.00934537],
       [ 0.044608  , -0.02808302, -0.05887076,  0.08452257,  0.01398618,
        -0.0686989 ,  0.09192241,  0.09783023

In [15]:
n=indata.phenotypes.shape[0]

In [66]:
q=indata.phenotypes.shape[1]
p=indata.genotypes.shape[1]

In [17]:
H=np.kron(indata.relatedness,Cg)+np.kron(np.ones((n,n)),Cn)

In [18]:
#Hinv=np.linalg.inv(H)

In [69]:
n

1000

In [23]:
import torch
H_cpu = torch.tensor(H, device='cpu')
Hinv_cpu = torch.inverse(H_cpu)
Hinv_cpu_s = torch.cholesky(Hinv_cpu)

In [24]:
Hinv_s = Hinv_cpu_s.numpy()

In [33]:
Hinv_s

array([[ 7.65718631e+01,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.74513848e+01,  4.12378584e+01,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.42864283e+01,  5.99904897e+01,  4.44980547e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 1.30020525e-02, -4.35135960e-02,  4.24132955e-02, ...,
         1.13756304e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.18439438e-02, -2.70886917e-02,  1.80370636e-02, ...,
         5.80120363e-01,  1.08340918e+00,  0.00000000e+00],
       [ 9.13057405e-02,  3.19908321e-02, -5.83037914e-02, ...,
         1.33612036e-01,  4.16071513e-01,  1.00031168e+00]])

In [39]:
Hinv_s.shape

(10000, 10000)

In [47]:
Y=indata.phenotypes.to_numpy()
y=np.matrix.flatten(Y,'F')

In [55]:
x= indata.genotypes.to_numpy()
X= np.kron(np.identity(q),x)

In [58]:
ytilde=Hinv_s @ y
Xtilde=Hinv_s @ X

In [59]:
Xtilde.shape

(10000, 200)

In [62]:
#from glmnet import glmnet, glmnetSet

In [64]:
from sklearn import linear_model
clf=linear_model.Lasso(alpha=0.1,fit_intercept=False)
clf.fit(X=Xtilde,y=ytilde)

Lasso(alpha=0.1, fit_intercept=False)

In [65]:
print(clf.coef_)

[ 0.05503209 -0.07023315 -0.         -0.17351879  0.16084125 -0.00826001
 -0.00208571 -0.05452381 -0.00482707  0.07513887 -0.02855315 -0.24322986
  0.17938708  0.12611488 -0.14450555 -0.03799008  0.03258278  0.21373953
  0.11099727 -0.25649907  0.03549984 -0.03091668 -0.12846505 -0.11322337
  0.24166141 -0.06271374  0.19117842  0.00907389 -0.00387544 -0.04902782
 -0.05987961  0.47890961 -0.23971922  0.08814588 -0.17917139  0.03348006
  0.19144489 -0.18444629  0.21011944 -0.31271743 -0.08265286  0.08867074
 -0.03119253  0.05775535 -0.21054143  0.09867941  0.02996047 -0.035046
  0.00403954 -0.15881513  0.24583397  0.09910783  0.33734139 -0.10743604
  0.06943324 -0.06556361  0.10024557  0.01906536 -0.0639536   0.16522135
 -0.04961631 -0.0989367   0.0133134   0.06008094 -0.08116815  0.03072792
 -0.2632546   0.11649148  0.01422528  0.06742333 -0.1096657  -0.05069834
 -0.00448895  0.07745685  0.19966074  0.02795085  0.50341123 -0.13828603
 -0.01705593 -0.00371651  0.1020203  -0.05229147 -0.1

In [71]:
beta=np.reshape(clf.coef_,(p,q),'F')

In [75]:
newY=Y-x@beta

In [79]:
#update residuals for new cova estimation step
indata.phenotypes= pd.DataFrame(newY, index=indata.pheno_samples,
                                       columns=indata.phenotype_ID)

In [ ]:
#need to transform X and Y based on estimated Cn and Ce

estimator = model.MultiTaskLR()

# estimate weights
theta_hat = estimator.individual_lasso(X, Y)
WW, L = estimator.give_weights(theta_hat)

# training
Theta= estimator.MTLR(X, Y, WW, L, C1=1000, C2=20, gamma=0.003)

In [ ]:
datalimmbo = LiMMBo(datainput=indata,
                            S=options.S,
                            timing=options.timing,
                            iterations=options.iterations,
                            verbose=options.verbose)
resultsBS = datalimmbo.runBootstrapCovarianceEstimation(seed=options.seed, cpus=options.cpus,minCooccurrence=options.minCooccurrence,n=options.runs)
resultsCovariance = datalimmbo.combineBootstrap(results=resultsBS)